In [2]:
pip install -Uq langchain langchain-community langchain-chroma langchain-groq langchain-openai

In [ ]:
pip install pypdf

In [7]:
import os
from google.colab import userdata


In [8]:
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [10]:
%ls

sample_data/  SaranAILabs2.pdf


In [11]:
loader = PyPDFLoader("SaranAILabs2.pdf")
docs = loader.load()

In [12]:
len(docs)

4

In [16]:
print(docs[3].page_content)

 
Page 10: Contact Information 
For any queries or support, you can reach out to the following: 
1. HR Department: hr@saranailabs.com 
2. IT Support: it-support@saranailabs.com 
3. General Queries: contact@saranailabs.com 
Office Address: Saran AI Labs Noida, Uttar Pradesh, India 
Let’s innovate and grow together! Welcome aboard! 
 
 


In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [18]:
len(splits)

12

In [20]:
print(splits[0].page_content)

Welcome to Saran AI Labs 
 
Page 1: About Saran AI Labs 
Saran AI Labs is a pioneering organization dedicated to advancing the field of Generative AI and its 
applications. Founded with the mission to innovate and empower businesses through cutting-edge 
AI solutions, we focus on: 
• Building AI-driven applications for diverse industries. 
• Researching and developing state-of-the-art machine learning models. 
• Empowering individuals and organizations with AI education and consulting.


In [21]:
print(splits[1].page_content)

• Empowering individuals and organizations with AI education and consulting. 
Our headquarters is located in Noida, India, and we serve clients worldwide. At Saran AI Labs, we 
pride ourselves on fostering a collaborative and innovative environment for our employees and 
partners. 
 
Page 2: Our Vision and Mission 
Vision: To be a global leader in Generative AI technology, transforming ideas into impactful solutions. 
Mission: 
• Deliver exceptional AI-powered solutions to our clients.


In [22]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [23]:
retreiver = vectorstore.as_retriever()

In [24]:
response = retreiver.invoke("what is SaranAI?")

In [27]:
print(response[0].page_content)

Welcome to Saran AI Labs 
 
Page 1: About Saran AI Labs 
Saran AI Labs is a pioneering organization dedicated to advancing the field of Generative AI and its 
applications. Founded with the mission to innovate and empower businesses through cutting-edge 
AI solutions, we focus on: 
• Building AI-driven applications for diverse industries. 
• Researching and developing state-of-the-art machine learning models. 
• Empowering individuals and organizations with AI education and consulting.


In [28]:
from langchain_core.prompts import ChatPromptTemplate

# Define a custom prompt template
prompt_template = """
You are an onboarding assistant for Saran AI labs that answers questions based on the context provided below.

Context:
{context}

Question: {question}

Please provide a detailed and accurate answer based on the given context.
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

In [29]:
# Function to format the retrieved documents for the prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
from langchain_groq import ChatGroq

In [33]:
llm = ChatGroq(model='llama-3.3-70b-versatile')

In [36]:
# Define the RAG chain
rag_chain = (
      {
          "context": retreiver | format_docs,
          "question": RunnablePassthrough()
      }
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
resp = rag_chain.invoke("tell me about Saran AI labs")

In [39]:
print(resp)

Welcome to Saran AI Labs. I'd be happy to provide you with an overview of our organization.

Saran AI Labs is a pioneering organization that is dedicated to advancing the field of Generative AI and its applications. Our mission is multifaceted, with a focus on:

1. **Building AI-driven applications**: We develop cutting-edge AI solutions for diverse industries, aiming to empower businesses through innovative technology.
2. **Research and development**: Our team is committed to researching and developing state-of-the-art machine learning models, pushing the boundaries of what is possible in the field of AI.
3. **Education and consulting**: We empower individuals and organizations by providing AI education and consulting services, helping them to harness the power of AI and stay ahead of the curve.

Our core mission is built around three key objectives:

* **Delivering exceptional AI-powered solutions**: We strive to provide our clients with the best possible AI solutions, tailored to th